# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f0d57dd8310>
├── 'a' --> tensor([[-0.8180,  0.8208,  0.2476],
│                   [ 0.2868, -1.9829,  1.9345]])
└── 'x' --> <FastTreeValue 0x7f0d54dbee50>
    └── 'c' --> tensor([[ 0.9174, -0.6179,  0.7713,  0.0928],
                        [-1.1549, -0.0906, -0.3709, -0.6765],
                        [ 1.3836,  0.0534,  0.0735, -0.1756]])

In [4]:
t.a

tensor([[-0.8180,  0.8208,  0.2476],
        [ 0.2868, -1.9829,  1.9345]])

In [5]:
%timeit t.a

72.4 ns ± 0.0586 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f0d57dd8310>
├── 'a' --> tensor([[ 1.4538,  1.5476, -0.4295],
│                   [ 0.3205,  1.2057, -1.0217]])
└── 'x' --> <FastTreeValue 0x7f0d54dbee50>
    └── 'c' --> tensor([[ 0.9174, -0.6179,  0.7713,  0.0928],
                        [-1.1549, -0.0906, -0.3709, -0.6765],
                        [ 1.3836,  0.0534,  0.0735, -0.1756]])

In [7]:
%timeit t.a = new_value

71.8 ns ± 0.099 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.8180,  0.8208,  0.2476],
               [ 0.2868, -1.9829,  1.9345]]),
    x: Batch(
           c: tensor([[ 0.9174, -0.6179,  0.7713,  0.0928],
                      [-1.1549, -0.0906, -0.3709, -0.6765],
                      [ 1.3836,  0.0534,  0.0735, -0.1756]]),
       ),
)

In [10]:
b.a

tensor([[-0.8180,  0.8208,  0.2476],
        [ 0.2868, -1.9829,  1.9345]])

In [11]:
%timeit b.a

65.6 ns ± 0.0604 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.5033,  0.5210, -1.0734],
               [-1.0285,  0.1065,  1.2796]]),
    x: Batch(
           c: tensor([[ 0.9174, -0.6179,  0.7713,  0.0928],
                      [-1.1549, -0.0906, -0.3709, -0.6765],
                      [ 1.3836,  0.0534,  0.0735, -0.1756]]),
       ),
)

In [13]:
%timeit b.a = new_value

543 ns ± 0.106 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

901 ns ± 1.05 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11 µs ± 32.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

145 µs ± 505 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

146 µs ± 607 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f0c88091160>
├── 'a' --> tensor([[[-0.8180,  0.8208,  0.2476],
│                    [ 0.2868, -1.9829,  1.9345]],
│           
│                   [[-0.8180,  0.8208,  0.2476],
│                    [ 0.2868, -1.9829,  1.9345]],
│           
│                   [[-0.8180,  0.8208,  0.2476],
│                    [ 0.2868, -1.9829,  1.9345]],
│           
│                   [[-0.8180,  0.8208,  0.2476],
│                    [ 0.2868, -1.9829,  1.9345]],
│           
│                   [[-0.8180,  0.8208,  0.2476],
│                    [ 0.2868, -1.9829,  1.9345]],
│           
│                   [[-0.8180,  0.8208,  0.2476],
│                    [ 0.2868, -1.9829,  1.9345]],
│           
│                   [[-0.8180,  0.8208,  0.2476],
│                    [ 0.2868, -1.9829,  1.9345]],
│           
│                   [[-0.8180,  0.8208,  0.2476],
│                    [ 0.2868, -1.9829,  1.9345]]])
└── 'x' --> <FastTreeValue 0x7f0d57e52eb0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.4 µs ± 99.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f0c87f988e0>
├── 'a' --> tensor([[-0.8180,  0.8208,  0.2476],
│                   [ 0.2868, -1.9829,  1.9345],
│                   [-0.8180,  0.8208,  0.2476],
│                   [ 0.2868, -1.9829,  1.9345],
│                   [-0.8180,  0.8208,  0.2476],
│                   [ 0.2868, -1.9829,  1.9345],
│                   [-0.8180,  0.8208,  0.2476],
│                   [ 0.2868, -1.9829,  1.9345],
│                   [-0.8180,  0.8208,  0.2476],
│                   [ 0.2868, -1.9829,  1.9345],
│                   [-0.8180,  0.8208,  0.2476],
│                   [ 0.2868, -1.9829,  1.9345],
│                   [-0.8180,  0.8208,  0.2476],
│                   [ 0.2868, -1.9829,  1.9345],
│                   [-0.8180,  0.8208,  0.2476],
│                   [ 0.2868, -1.9829,  1.9345]])
└── 'x' --> <FastTreeValue 0x7f0d54dc4490>
    └── 'c' --> tensor([[ 0.9174, -0.6179,  0.7713,  0.0928],
                        [-1.1549, -0.0906, -0.3709, -0.6765],
                 

In [23]:
%timeit t_cat(trees)

31.4 µs ± 36.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

66.6 µs ± 170 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.8180,  0.8208,  0.2476],
                [ 0.2868, -1.9829,  1.9345]],
       
               [[-0.8180,  0.8208,  0.2476],
                [ 0.2868, -1.9829,  1.9345]],
       
               [[-0.8180,  0.8208,  0.2476],
                [ 0.2868, -1.9829,  1.9345]],
       
               [[-0.8180,  0.8208,  0.2476],
                [ 0.2868, -1.9829,  1.9345]],
       
               [[-0.8180,  0.8208,  0.2476],
                [ 0.2868, -1.9829,  1.9345]],
       
               [[-0.8180,  0.8208,  0.2476],
                [ 0.2868, -1.9829,  1.9345]],
       
               [[-0.8180,  0.8208,  0.2476],
                [ 0.2868, -1.9829,  1.9345]],
       
               [[-0.8180,  0.8208,  0.2476],
                [ 0.2868, -1.9829,  1.9345]]]),
    x: Batch(
           c: tensor([[[ 0.9174, -0.6179,  0.7713,  0.0928],
                       [-1.1549, -0.0906, -0.3709, -0.6765],
                       [ 1.3836,  0.0534,  0.0735, -0.1756]],
         

In [26]:
%timeit Batch.stack(batches)

83.7 µs ± 326 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.8180,  0.8208,  0.2476],
               [ 0.2868, -1.9829,  1.9345],
               [-0.8180,  0.8208,  0.2476],
               [ 0.2868, -1.9829,  1.9345],
               [-0.8180,  0.8208,  0.2476],
               [ 0.2868, -1.9829,  1.9345],
               [-0.8180,  0.8208,  0.2476],
               [ 0.2868, -1.9829,  1.9345],
               [-0.8180,  0.8208,  0.2476],
               [ 0.2868, -1.9829,  1.9345],
               [-0.8180,  0.8208,  0.2476],
               [ 0.2868, -1.9829,  1.9345],
               [-0.8180,  0.8208,  0.2476],
               [ 0.2868, -1.9829,  1.9345],
               [-0.8180,  0.8208,  0.2476],
               [ 0.2868, -1.9829,  1.9345]]),
    x: Batch(
           c: tensor([[ 0.9174, -0.6179,  0.7713,  0.0928],
                      [-1.1549, -0.0906, -0.3709, -0.6765],
                      [ 1.3836,  0.0534,  0.0735, -0.1756],
                      [ 0.9174, -0.6179,  0.7713,  0.0928],
                      [-1.1549, -

In [28]:
%timeit Batch.cat(batches)

154 µs ± 268 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

351 µs ± 6.21 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
